In [ ]:
#| default_exp utils/basic

## Plotting

In [ ]:
#| export
import matplotlib.pyplot as plt

In [ ]:
#| export
def savefig(name, **kwargs):
    plt.savefig(f"../figures/{name}.png", bbox_inches="tight", **kwargs)
    plt.savefig(f"../figures/{name}.pdf", bbox_inches="tight", **kwargs)

## Utilities functions

In [ ]:
#| export
import os
import requests

import polars as pl
import polars.selectors as cs

import pandas as pd
import xarray as xr

import pandas
import numpy as np
from xarray_einstats import linalg

from datetime import timedelta

from loguru import logger
from multipledispatch import dispatch

from xarray import DataArray
from typing import Union, Collection, Callable, Optional, Tuple
from typing import Any, Dict

In [ ]:
#| export
from pipe import select
from fastcore.utils import partial

In [ ]:
#| export
def pmap(func, *args, **kwargs):
    """
    map with `partial`
    """
    return select(partial(func, *args, **kwargs))

In [ ]:
#| export
def get_code(data, name):
    import lineapy
    "use lineapy to get code from data"
    lineapy.save(data, name)
    code = lineapy.get(name).get_code()
    print(code)
    return code

## `Kedro`

Standard import
```{python}
from kedro.pipeline import Pipeline, node
from kedro.pipeline.modular_pipeline import pipeline

def create_pipeline(**kwargs) -> Pipeline:
    return pipeline([
        ...
    ])
```

In [ ]:
#| export
from kedro.config import OmegaConfigLoader
from kedro.io import DataCatalog

def load_catalog(conf_source: str = "../conf", catalog_source: str = "catalog"):
    # Initialise a ConfigLoader
    conf_loader = OmegaConfigLoader(conf_source)

    # Load the data catalog configuration from catalog.yml
    conf_catalog = conf_loader.get(catalog_source)

    # Create the DataCatalog instance from the configuration
    catalog = DataCatalog.from_config(conf_catalog)
    
    return catalog

The following `load_catalog` provides project-aware access to the catalog. The preceding `load_catalog` only works when notebook is run from the project root.

In [ ]:
#| export
from kedro.framework.session import KedroSession
from kedro.framework.startup import bootstrap_project
from kedro.ipython import _resolve_project_path

def load_catalog(project_path: str = '../'):
    project_path = _resolve_project_path(project_path)
    metadata = bootstrap_project(project_path)
    # configure_project(metadata.package_name)

    session = KedroSession.create(
        metadata.package_name, project_path,
    )
    context = session.load_context()
    catalog = context.catalog
    return catalog

#### Test

In [ ]:
catalog = load_catalog()
catalog.list()
catalog.load('raw_jno_mswim2d')

[10/13/23 10:32:57] WARNING  /Users/zijin/mambaforge/envs/cool_planet/lib/python3.10/site-packages/ ]8;id=256448;file:///Users/zijin/mambaforge/envs/cool_planet/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=968118;file:///Users/zijin/mambaforge/envs/cool_planet/lib/python3.10/warnings.py#109\109]8;;\
                             kedro/io/data_catalog.py:300: FutureWarning: Defining the 'layer'                     
                             attribute at the top level is deprecated and will be removed in Kedro                 
                             0.19.0. Please move 'layer' inside the 'metadata' -> 'kedro-viz'                      
                             attributes. See                                                                       
                             https://docs.kedro.org/en/latest/visualisation/kedro-viz_visualisation                
                             .html#visualise-layers for more information.                                          
                               warnings.warn(                                                                      
                                                                                                                   

                    INFO     Loading data from 'raw_jno_mswim2d' (CSVDataSet)...                ]8;id=11697;file:///Users/zijin/mambaforge/envs/cool_planet/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=70873;file:///Users/zijin/mambaforge/envs/cool_planet/lib/python3.10/site-packages/kedro/io/data_catalog.py#492\492]8;;\

,Date_Time,hour,r,phi,rho,ux,uy,uz,bx,by,bz,Ti
0,2011-08-25T00:00:00,0.0,1.004,258.679,2.77562,-85.5,-434.4,-0.0,2.9100,0.6560,0.5780,76394
1,2011-08-25T01:00:00,1.0,1.003,258.735,2.82988,-92.4,-430.8,-0.0,2.8200,0.5610,0.5230,76600
2,2011-08-25T02:00:00,2.0,1.003,258.791,2.94476,-93.1,-431.2,-0.0,2.7300,0.1950,0.9490,74014
3,2011-08-25T03:00:00,3.0,1.004,258.858,2.89843,-89.1,-430.0,0.0,2.8400,0.1520,1.1800,70426
4,2011-08-25T04:00:00,4.0,1.004,258.914,2.71252,-84.0,-429.3,-0.0,3.1200,0.3080,1.0800,68131
...,...,...,...,...,...,...,...,...,...,...,...,...
42380,2016-06-29T20:00:00,692.0,5.410,101.212,0.02956,-96.9,460.9,-0.0,0.0865,0.0240,0.0622,24063
42381,2016-06-29T21:00:00,693.0,5.410,101.212,0.02992,-96.7,460.0,-0.0,0.0876,0.0239,0.0628,23837
42382,2016-06-29T22:00:00,694.0,5.410,101.212,0.03026,-96.5,459.1,-0.0,0.0887,0.0237,0.0634,23619
42383,2016-06-29T23:00:00,695.0,5.410,101.212,0.03057,-96.4,458.3,-0.0,0.0900,0.0235,0.0640,23409


## `Polars`

In [ ]:
#| export
from fastcore.utils import patch

In [ ]:
#| export
import pycdfpp

In [ ]:
#| export
def cdf2pl(file_path: str, var_name: str) -> pl.LazyFrame:
    "Convert a CDF file to Polars Dataframe"
    cdf = pycdfpp.load(file_path)
    epoch_time = pycdfpp.to_datetime64(cdf["Epoch"])
    var_values = cdf[var_name].values

    # Dynamically create column names based on the shape of the field values
    columns = {
        "time": epoch_time,
        **{f"{var_name}_{i}": var_values[:, i] for i in range(var_values.shape[1])}
    }
    
    df = pl.DataFrame(columns).lazy()
    return df


In [ ]:
#| export
@patch
def plot(self:pl.DataFrame, *args, **kwargs):
    return self.to_pandas().plot(*args, **kwargs)

In [ ]:
#| export
def _expand_selectors(items: Any, *more_items: Any) -> list[Any]:
    """
    See `_expand_selectors` in `polars`.
    """
    expanded: list[Any] = []
    for item in (
        *(
            items
            if isinstance(items, Collection) and not isinstance(items, str)
            else [items]
        ),
        *more_items,
    ):
        expanded.append(item)
    return expanded

def pl_norm(columns, *more_columns) -> pl.Expr:
    """
    Computes the square root of the sum of squares for the given columns.

    Args:
    *columns (str): Names of the columns.

    Returns:
    pl.Expr: Expression representing the square root of the sum of squares.
    """
    all_columns = _expand_selectors(columns, *more_columns)
    squares = [pl.col(column).pow(2) for column in all_columns]

    return sum(squares).sqrt()

#### Partition the dataset by year

In [ ]:
#| export
def partition_data_by_year(df: pl.DataFrame) -> Dict[str, pl.DataFrame]:
    """Partition the dataset by year

    Args:
        df: Input DataFrame.

    Returns:
        Partitioned DataFrame.
    """
    return (
        df.with_columns(year=pl.col("time").dt.year().cast(pl.Utf8))
        .collect()
        .partition_by("year", include_key=False, as_dict=True)
    )

In [ ]:
#| export
def concat_partitions(partitioned_input: Dict[str, Callable]) -> pandas.DataFrame:
    """Concatenate input partitions into one pandas DataFrame.

    Args:
        partitioned_input: A dictionary with partition ids as keys and load functions as values.

    Returns:
        Pandas DataFrame representing a concatenation of all loaded partitions.
    """
    partitions_data = [
        partition_load_func() for partition_load_func in partitioned_input.values()
    ]  # load the actual partition data
    result = pandas.concat(partitions_data, ignore_index=True, sort=True)
    return result

#### Resample data

In [ ]:
#| export
def format_timedelta(time):
    """Format timedelta to `timedelta`"""
    if isinstance(time, timedelta):
        return time
    elif isinstance(time, str):
        return pd.Timedelta(time)
    elif isinstance(time, int):
        return pd.Timedelta(seconds=time)
    else:
        raise TypeError(f"Unsupported type: {type(time)}")

def resample(
    df: pl.DataFrame | pl.LazyFrame, 
    every: timedelta | str | int, period: str | timedelta = None,
    time_column='time',
) -> pl.DataFrame | pl.LazyFrame:
    """Resample the DataFrame"""
    if period is None:
        period = every
    every = format_timedelta(every)
    period = format_timedelta(period)
    return (
        df.sort(time_column)
        .group_by_dynamic(time_column, every=every, period=period)
        .agg(cs.numeric().mean())
        .with_columns(
            (pl.col(time_column) + period / 2).dt.cast_time_unit("ns")
        )
    )

#### Read `lbl` file

In [ ]:
#| export
from pathlib import PurePosixPath
import fsspec

from kedro.io import AbstractDataset
from kedro.io.core import get_filepath_str, get_protocol_and_path
from kedro.extras.datasets.pandas import CSVDataSet

import pdr

In [ ]:
#| export
HTTP_PROTOCOLS = ("http", "https")

class LblDataset(AbstractDataset):
    def __init__(
        self, filepath: str,
        load_type: str = 'table',
        metadata: Dict[str, Any] = None,
        ):
        # parse the path and protocol (e.g. file, http, s3, etc.)
        protocol, path = get_protocol_and_path(filepath)
        self._protocol = protocol
        self._filepath = PurePosixPath(path)
        
        self._fs = fsspec.filesystem(self._protocol)

        self.load_type = load_type
        self.metadata = metadata

    def _load(self):
        # using get_filepath_str ensures that the protocol and path are appended correctly for different filesystems
        load_path = get_filepath_str(self._filepath, self._protocol)

        if self._protocol in HTTP_PROTOCOLS:
            import pooch            
            local_fp = pooch.retrieve(load_path, known_hash=None)
        else:
            local_fp = load_path

        if self.load_type == 'table':
            df = pdr.read(local_fp).TABLE # NOTE: Support for read from url is not currently implemented.
        elif self.load_type == 'index':
            df = pandas.read_csv(local_fp, delimiter=",", quotechar='"')
            df.columns = df.columns.str.replace(" ", "")
            
        return df

    def _save(self):
        pass
    
    def _describe(self):
        """Returns a dict that describes the attributes of the dataset."""
        return dict(filepath=self._filepath, protocol=self._protocol)

In [ ]:
#| export
from humanize import naturalsize

In [ ]:
#| export
def get_memory_usage(data):
    datatype = type(data)
    match datatype:
        case pl.DataFrame:
            size = data.estimated_size()
        case pd.DataFrame:
            size = data.memory_usage().sum()
        case xr.DataArray:
            size = data.nbytes

    logger.info(f"{naturalsize(size)} ({datatype.__name__})")
    return size

In [ ]:
#| export
def download_file(url, local_dir="./", file_name=None):
    """
    Download a file from a URL and save it locally.

    Returns:
    file_path (str): Path to the downloaded file.
    """
    if file_name is None:
        file_name = url.split("/")[-1]

    file_path = os.path.join(local_dir, file_name)
    dir = os.path.dirname(file_path)
    if not os.path.isdir(dir):
        os.makedirs(dir)

    if not os.path.exists(file_path):
        logger.debug(f"Downloading from {url}")
        response = requests.get(url)
        with open(file_path, "wb") as f:
            f.write(response.content)
    return file_path

def check_fgm(vec):
    # check if time is monotonic increasing
    logger.info("Check if time is monotonic increasing")
    assert vec.time.to_series().is_monotonic_increasing
    # check available time difference
    logger.info(
        f"Available time delta: {vec.time.diff(dim='time').to_series().unique()}"
    )
    # data_array.time.diff(dim="time").plot(yscale="log")


def col_renamer(lbl: str):
    if lbl.startswith("BX"):
        return "BX"
    if lbl.startswith("BY"):
        return "BY"
    if lbl.startswith("BZ"):
        return "BZ"
    return lbl


def df2ts(
    df: Union[pandas.DataFrame, pl.DataFrame, pl.LazyFrame], cols, attrs=None, name=None
):
    for col in cols:
        if col not in df.columns:
            raise KeyError(f"Expected column {col} not found in the dataframe.")

    if isinstance(df, pl.LazyFrame):
        df = df.collect()

    # Prepare data
    data = df[cols]

    # Prepare coordinates
    time = df.index if isinstance(df, pandas.DataFrame) else df["time"]

    # Create the DataArray
    coords = {"time": time, "v_dim": cols}

    return xr.DataArray(data, coords=coords, attrs=attrs, name=name)


def sat_get_fgm_from_df(df: Union[pandas.DataFrame, pl.DataFrame, pl.LazyFrame]):
    attrs = {"coordinate_system": "se", "units": "nT"}

    return df2ts(df, cols=["BX", "BY", "BZ"], attrs=attrs, name="sat_fgm")


def juno_get_state(df: Union[pandas.DataFrame, pl.DataFrame, pl.LazyFrame]):
    attrs = {"coordinate_system": "se", "units": "km"}
    return df2ts(df, cols=["X", "Y", "Z"], attrs=attrs, name="sat_state")


def calc_vec_mag(vec) -> DataArray:
    return linalg.norm(vec, dims="v_dim")

In [ ]:
#| export
@dispatch(pl.DataFrame)
def calc_time_diff(data: pl.DataFrame): 
    return data.get_column('time').diff(null_behavior="drop").unique().sort()

@dispatch(pl.LazyFrame)
def calc_time_diff(
    data: pl.LazyFrame
) -> pl.Series: 
    return calc_time_diff(data.collect())

## Obsolete codes

In [ ]:
from flox.xarray import xarray_reduce

In [ ]:
#| hide
# Helper function to calculate combined standard deviation
def calc_combined_std(col_name):
    return (
        pl.concat_list([pl.col(col_name).shift(-2), pl.col(col_name).shift(2)])
        .list.eval(pl.element().std())
        .flatten()
        .alias(f"{col_name}_combined_std")
    )

In [ ]:
@dispatch(xr.DataArray, object)
def compute_index_std(data: DataArray, tau):
    """
    Examples
    --------
    >>> i1 = index_std(juno_fgm_b, tau)
    """

    # NOTE: large tau values will speed up the computation

    # Resample the data based on the provided time interval.
    grouped_data = data.resample(time=pandas.Timedelta(tau, unit="s"))

    # Compute the standard deviation for all groups
    vec_stds = linalg.norm(grouped_data.std(dim="time"), dims="v_dim")
    # vec_stds = grouped_data.map(calc_vec_std) # NOTE: This is way much slower (like 30x slower)

    offset = pandas.Timedelta(tau / 2, unit="s")
    vec_stds["time"] = vec_stds["time"] + offset

    vec_stds_next = vec_stds.assign_coords(
        {"time": vec_stds["time"] - pandas.Timedelta(tau, unit="s")}
    )
    vec_stds_previous = vec_stds.assign_coords(
        {"time": vec_stds["time"] + pandas.Timedelta(tau, unit="s")}
    )
    return np.minimum(vec_stds / vec_stds_next, vec_stds / vec_stds_previous)

In [ ]:
#| hide
def _compute_index_fluctuation_old(df: pl.DataFrame, tau) -> pl.DataFrame:
    """
    Compute the fluctuation index based on the given DataFrame, and tau value.

    Parameters:
    - df (pl.DataFrame): The input DataFrame.
    - tau (int): The time interval value.

    Returns:
    - pl.DataFrame: DataFrame with calculated 'index_fluctuation' column.

    Examples
    --------
    >>> result_df = compute_index_fluctuation(df, tau)
    """
    if isinstance(tau, (int, float)):
        tau = timedelta(seconds=tau)

    # Group and compute standard deviations
    group_df = (
        df.group_by_dynamic("time", every=tau / 2, period=tau)
        .agg(
            pl.col(["BX", "BY", "BZ"]),
            pl.col("BX").std().alias("BX_std"),
            pl.col("BY").std().alias("BY_std"),
            pl.col("BZ").std().alias("BZ_std"),
        )
        .with_columns(
            pl_norm("BX_std", "BY_std", "BZ_std").alias("B_std"),
        )
        .drop("BX_std", "BY_std", "BZ_std")
    )

    # Compute fluctuation index
    index_fluctuation_df = (
        group_df.with_columns(
            calc_combined_std("BX"),
            calc_combined_std("BY"),
            calc_combined_std("BZ"),
        )
        .drop("BX", "BY", "BZ")
        .with_columns(
            pl_norm("BX_combined_std", "BY_combined_std", "BZ_combined_std").alias(
                "B_combined_std"
            ),
            pl.sum_horizontal(
                pl.col("B_std").shift(-2), pl.col("B_std").shift(2)
            ).alias("B_added_std"),
        )
        .drop("BX_combined_std", "BY_combined_std", "BZ_combined_std")
        .with_columns(
            (pl.col("B_combined_std") / pl.col("B_added_std")).alias(
                "index_fluctuation"
            ),
        )
    )

    return index_fluctuation_df

    # NOTE: the following code is about 2x slower than the above code
    # group_df.with_columns(
    #     pl.concat_list([pl.col("BX_group").shift(-2), pl.col("BX_group").shift(2)]),
    # ).explode("BX_group").sort("time").group_by("time").agg(
    #     pl.col("BX_group").std().alias("BX_combined_std"),
    # )

    # NOTE: the following code is about 2x slower than the above code
    # pl.concat(
    #     [
    #         group_df.with_columns(pl.col("BX_group").shift(-2)).explode("BX_group"),
    #         group_df.with_columns(pl.col("BX_group").shift(2)).explode("BX_group"),
    #     ]
    # ).sort("time").group_by("time").agg(
    #     pl.col("BX_group").std().alias("BX_combined_std"),
    # )


# NOTE: the two implementation of computing the fluctuation are equivalent, but the following one is about a little bit slower.
def _compute_index_fluctuation_xr(data: xr.DataArray, tau):
    # Resample the data based on the provided time interval.
    grouped_data = data.resample(time=pandas.Timedelta(tau, unit="s"))

    # Pre-compute std for all groups
    vec_stds = linalg.norm(grouped_data.std(dim="time"), dims="v_dim")

    fluctuation_values = []
    group_keys = list(grouped_data.groups.keys())

    # Iterate over the groups, skipping the first and the last one.
    for i in range(1, len(group_keys) - 1):
        prev_std = vec_stds[i - 1]
        next_std = vec_stds[i + 1]

        prev_group_indices = grouped_data.groups[group_keys[i - 1]]
        next_group_indices = grouped_data.groups[group_keys[i + 1]]
        prev_group = data[prev_group_indices]
        next_group = data[next_group_indices]

        combined_group = xr.concat([prev_group, next_group], dim="time")
        combined_std = calc_vec_std(combined_group)

        fluctuation = combined_std / (prev_std + next_std)
        fluctuation_values.append(fluctuation)
    return DataArray(fluctuation_values, dims=["time"])


def compute_index_fluctuation(data, tau):
    """helper function to compute fluctuation index

    Notes: the results returned are a little bit different for the two implementations (because of the implementation of `std`).
    """

    if isinstance(data, pl.DataFrame):
        return _compute_index_fluctuation_old(data, tau)
    if isinstance(data, xr.DataArray):
        return compute_index_fluctuation_xr(data, tau)

def compute_index_fluctuation_xr(data: xr.DataArray, tau: int) -> xr.DataArray:
    """
    Computes the fluctuation index for a given data array based on a specified time interval.

    Parameters:
    - data: The xarray DataArray containing the data to be processed.
    - tau: Time interval in seconds for resampling.

    Returns:
    - fluctuation: xarray DataArray containing the fluctuation indices.

    Notes
    -----
        ddof=0 is used for calculating the standard deviation. (ddof=1 is for sample standard deviation)
    """

    # Resample the data based on the provided time interval.
    grouped_data = data.resample(time=pandas.Timedelta(tau, unit="s"))

    # Pre-compute the standard deviation for all groups
    vec_stds = linalg.norm(grouped_data.std(dim="time"), dims="v_dim")

    # Assign coordinates for pre and next groups based on time offset
    offset = pandas.Timedelta(tau, unit="s")
    pre_stds = vec_stds.assign_coords({"time": vec_stds["time"] - offset})
    next_stds = vec_stds.assign_coords({"time": vec_stds["time"] + offset})

    # Offset the keys of the group dictionary to get previous and next groups
    groups_dict = grouped_data.groups

    # Create DataArrays for previous and next time labels using the slices from the groups dictionaries
    prev_labels = xr.concat(
        [
            xr.DataArray(
                key + offset,
                dims=["time"],
                coords={"time": data.time[slice]},
                name="time",
            )
            for key, slice in groups_dict.items()
        ],
        dim="time",
    )
    next_labels = xr.concat(
        [
            xr.DataArray(
                key - offset,
                dims=["time"],
                coords={"time": data.time[slice]},
                name="time",
            )
            for key, slice in groups_dict.items()
        ],
        dim="time",
    )

    # Concatenate the previous and next labels into a single DataArray
    labels = xr.concat([prev_labels, next_labels], dim="y")

    # Compute the combined standard deviation for the data using the labels
    combined_stds = linalg.norm(xarray_reduce(data, labels, func="std"), dims="v_dim")

    # Calculate the fluctuation index
    fluctuation = combined_stds / (pre_stds + next_stds)
    fluctuation["time"] = fluctuation["time"] + pandas.Timedelta(tau / 2, unit="s")

    return fluctuation

In [ ]:
def index_diff(data: DataArray, tau):
    grouped_data = data.resample(time=pandas.Timedelta(tau, unit='s'))

    dvecs = grouped_data.first()-grouped_data.last()
    vec_mean_mags = grouped_data.map(calc_vec_mean_mag)
    vec_diffs = linalg.norm(dvecs, dims='v_dim') / vec_mean_mags
    
    # vec_diffs = grouped_data.map(calc_vec_relative_diff) # NOTE: this is slower than the above implementation.
    # INFO: Do your spatial and temporal indexing (e.g. .sel() or .isel()) early in the pipeline, especially before calling resample() or groupby(). Grouping and resampling triggers some computation on all the blocks, which in theory should commute with indexing, but this optimization hasn’t been implemented in Dask yet. (See Dask issue #746).
    
    offset = pandas.Timedelta(tau/2, unit='s')
    vec_diffs['time'] = vec_diffs['time'] + offset
    return vec_diffs


# %%
def _compute_indices_old(df: pl.DataFrame, tau):
    """
    Compute all index based on the given DataFrame and tau value.

    Parameters
    ----------
    - df (pl.DataFrame): The input DataFrame.
    - tau (int): The time interval value.

    Returns
    -------
    - tuple: Tuple containing DataFrame results for fluctuation index, standard deviation index, and 'index_num'.

    Examples
    --------
    >>> indices = compute_indices(df, tau)

    Notes
    -----
    Simply shift to calculate index_std would not work correctly if data is missing, like `std_next = pl.col("B_std").shift(-2)`.
    """

    if isinstance(tau, (int, float)):
        tau = timedelta(seconds=tau)

    b_cols = ["BX", "BY", "BZ"]
    db_cols = ["d" + col_name + "_vec" for col_name in b_cols]

    std_next = pl.col("B_std").shift(-2)
    std_current = pl.col("B_std")
    std_previous = pl.col("B_std").shift(2)
    pl_std_index = pl.min_horizontal(
        [std_current / std_previous, std_current / std_next]
    )

    # Compute fluctuation index
    group_df = (
        df.with_columns(pl_norm("BX", "BY", "BZ").alias("B"))
        .group_by_dynamic("time", every=tau / 2, period=tau)
        .agg(
            pl.count(),
            pl.col(b_cols),
            pl.col("BX").std().alias("BX_std"),
            pl.col("BY").std().alias("BY_std"),
            pl.col("BZ").std().alias("BZ_std"),
            pl.col("B").mean().alias("B_mean"),
            *pl_dvec(b_cols),
        )
        .with_columns(
            pl_norm("BX_std", "BY_std", "BZ_std").alias("B_std"),
            pl_norm(db_cols).alias("dB_vec"),
        )
        .drop("BX_std", "BY_std", "BZ_std")
    )

    indices = (
        group_df.with_columns(
            calc_combined_std("BX"),
            calc_combined_std("BY"),
            calc_combined_std("BZ"),
        )
        .drop("BX", "BY", "BZ")
        .with_columns(
            pl_norm("BX_combined_std", "BY_combined_std", "BZ_combined_std").alias(
                "B_combined_std"
            ),
            pl.sum_horizontal(
                pl.col("B_std").shift(-2), pl.col("B_std").shift(2)
            ).alias("B_added_std"),
        )
        .drop("BX_combined_std", "BY_combined_std", "BZ_combined_std")
        .with_columns(
            pl_std_index.alias("index_std"),
            (pl.col("B_combined_std") / pl.col("B_added_std")).alias(
                "index_fluctuation"
            ),
            (pl.col("dB_vec") / pl.col("B_mean")).alias("index_diff"),
        )
    )
    return indices